# 1. Sequence to Sequence Learning with Neural Networks
* Pytorch와 torchtext를 사용하여 sequence에서 sequence로 이동하는 NMT 모델을 구축하자!
* many to many를 푸는 모든 문제에 적용할 수 있는 구조에요

## Introduction
* 일반적인 Seq2Seq은 RNN을 사용하여 입력 문장을 단일 벡터로 인코딩하는 Enc-Dec 모델이에요
* 위 단일 벡터를 context vecor라고 부를게요
* context vector는 전체 입력 문장의 추상적인 표현이에요
* 이 벡터는 한 번에 한 단어씩 생성하여 대상 문장을 출력하는 방법을 학습하는 Decoder RNN에 의해 Decode되서 타켓 문장을 만들어내요

## Preparing Data

In [1]:
!python -m spacy download en
!python -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.9MB 8.3MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp37-none-any.whl size=14907055 sha256=4343b8309cbf5b7115ed2b2d126a9d11cd7c40fe76eda9232cf2fe50ad26c911
  Stored in directory: /tmp/pip-ephem-wheel-cache-tkpuen93/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext import data, datasets
from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [3]:
import torchtext
print(torchtext.__version__)

0.10.0


In [4]:
torch.__version__

'1.9.0+cu102'

In [5]:
spacy.__version__

'2.2.4'

In [6]:
np.__version__

'1.19.5'

## 시드 고정

In [7]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [8]:
import spacy

spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [9]:
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [10]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [11]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 1.09MB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 167kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 159kB/s]


In [12]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


In [13]:
print(vars(train_data.examples[0]))

{'src': ['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


In [14]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [15]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 7855
Unique tokens in target (en) vocabulary: 5893


In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

## Building the Seq2Seq Model



### LSTM default values

In [18]:
x = torch.randn(3, 5, 10) # (batch_size, seq_len, hidden_dim)

In [19]:
rnn = nn.LSTM(10, 5, batch_first=False, bidirectional=False, num_layers=1)
rnn

LSTM(10, 5)

In [20]:
output, (hidden, cell) = rnn(x.permute(1, 0, 2)) # permutation

In [21]:
output.size() # (seq_len, batch_size, hidden_dim)

torch.Size([5, 3, 5])

In [22]:
hidden.size(), cell.size() # (bidirect * num_layers, seq_len, hidden_dim)

(torch.Size([1, 3, 5]), torch.Size([1, 3, 5]))

#### LSTM batch_first=True

In [23]:
rnn = nn.LSTM(10, 5, batch_first=True, bidirectional=False, num_layers=1)
rnn

LSTM(10, 5, batch_first=True)

In [24]:
output, (hidden, cell) = rnn(x)

In [25]:
output.size() # (batch_size, seq_len, hidden_dim)

torch.Size([3, 5, 5])

In [26]:
hidden.size(), cell.size() # (bidirect * num_layers, seq_len, hidden_dim)

(torch.Size([1, 3, 5]), torch.Size([1, 3, 5]))

#### LSTM batch_first=True, num_layers=2

In [27]:
rnn = nn.LSTM(10, 5, batch_first=True, bidirectional=False, num_layers=2)
rnn

LSTM(10, 5, num_layers=2, batch_first=True)

In [28]:
output, (hidden, cell) = rnn(x)

In [29]:
output.size() # (batch_size, seq_len, hidden_dim)
              # layer만 높인거라 size에 변화 X

torch.Size([3, 5, 5])

In [30]:
hidden.size(), cell.size() # (bidirect * num_layers, seq_len, hidden_dim)
                           # layer별 결과도 가져오기 때문에 size up

(torch.Size([2, 3, 5]), torch.Size([2, 3, 5]))

#### LSTM batch_first=True, num_layers=2, bidirectional=True

In [31]:
rnn = nn.LSTM(10, 5, batch_first=True, bidirectional=True, num_layers=2)
rnn

LSTM(10, 5, num_layers=2, batch_first=True, bidirectional=True)

In [32]:
output, (hidden, cell) = rnn(x)

In [33]:
output.size() # (batch_size, seq_len, hidden_dim * 2)
              # forward and backward pass 둘을 concat해주기 때문

torch.Size([3, 5, 10])

In [34]:
hidden.size(), cell.size() # (bidirect * num_layers, seq_len, hidden_dim)

(torch.Size([4, 3, 5]), torch.Size([4, 3, 5]))

#### dropout은 n_layer>=2부터

In [35]:
nn.LSTM(10, 5, batch_first=True, num_layers=2, dropout=0.2)

LSTM(10, 5, num_layers=2, batch_first=True, dropout=0.2)

In [36]:
nn.LSTM(10, 5, batch_first=True, num_layers=1, dropout=0.2)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


LSTM(10, 5, batch_first=True, dropout=0.2)

In [37]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

#### Decoder

In [38]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        # lm_head
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

#### Seq2Seq

In [39]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

#### Training the seq2seq model

In [40]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [41]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [42]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 13,899,013 trainable parameters


In [43]:
optimizer = optim.Adam(model.parameters())

In [44]:
y_pred = torch.randn(10, 3, 30000) # (seq_len, bsz, vocab_size)

In [45]:
y_pred

tensor([[[-0.1591,  1.6328,  0.7653,  ...,  0.1014,  0.5516,  0.5493],
         [-2.0111, -0.4198, -0.0253,  ...,  0.8188,  0.0037, -0.5213],
         [-0.9654, -0.7851,  0.1644,  ..., -1.8628, -0.6396, -1.2193]],

        [[ 1.3900, -0.6168,  1.1222,  ..., -0.6201,  1.0073, -1.1057],
         [-1.0108, -0.9333,  0.1244,  ...,  0.6850,  0.2258, -0.1413],
         [ 1.8606,  0.1106, -0.2428,  ..., -0.1104,  1.4462,  1.2030]],

        [[-0.4314,  0.0143,  1.6798,  ...,  0.5633, -0.2121,  1.4145],
         [ 0.0843, -1.2277,  0.7508,  ...,  1.0950, -0.4211,  0.1849],
         [-0.9607,  0.7846, -0.9372,  ...,  0.5146, -0.7070,  0.0120]],

        ...,

        [[-0.1027,  0.6405,  0.7995,  ...,  0.1188,  1.1455,  1.1473],
         [-0.3918, -0.4921,  0.0326,  ..., -0.1471,  1.4715, -0.4273],
         [ 0.3269,  0.0294,  1.0668,  ...,  1.6670,  0.2816,  0.9254]],

        [[-1.6181, -0.4994, -0.4934,  ...,  0.1357,  0.6181,  1.0628],
         [-0.2782,  1.3450, -0.6268,  ...,  1.7034,  0.

In [46]:
y = torch.LongTensor(10, 3).random_(1, 29999)
y[-3:, 0] = 0
y[-2:, 2] = 0
y

tensor([[18165, 23600,  8779],
        [ 3636, 13822, 25639],
        [  648,    56, 26262],
        [28121, 19014, 16914],
        [ 2873, 28559, 18532],
        [18395,  4326, 13346],
        [  124, 18832,  8279],
        [    0,  6665, 28036],
        [    0, 24265,     0],
        [    0,  1216,     0]])

In [47]:
criterion = nn.CrossEntropyLoss(ignore_index=0)

In [48]:
criterion(y_pred.view(-1, 30000), y.view(-1))

tensor(10.9163)

In [49]:
nn.NLLLoss(ignore_index=0)(
    torch.log_softmax(y_pred.view(-1, 30000), dim=-1), 
    y.view(-1)
)

tensor(10.9163)

In [50]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [51]:
TRG_PAD_IDX

1

In [52]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [53]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [54]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [55]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 43s
	Train Loss: 4.846 | Train PPL: 127.288
	 Val. Loss: 4.931 |  Val. PPL: 138.522
Epoch: 02 | Time: 0m 43s
	Train Loss: 4.307 | Train PPL:  74.248
	 Val. Loss: 4.668 |  Val. PPL: 106.465
Epoch: 03 | Time: 0m 44s
	Train Loss: 3.983 | Train PPL:  53.662
	 Val. Loss: 4.490 |  Val. PPL:  89.114
Epoch: 04 | Time: 0m 43s
	Train Loss: 3.806 | Train PPL:  44.959
	 Val. Loss: 4.256 |  Val. PPL:  70.498
Epoch: 05 | Time: 0m 43s
	Train Loss: 3.611 | Train PPL:  36.999
	 Val. Loss: 4.132 |  Val. PPL:  62.310
Epoch: 06 | Time: 0m 43s
	Train Loss: 3.436 | Train PPL:  31.061
	 Val. Loss: 3.968 |  Val. PPL:  52.877
Epoch: 07 | Time: 0m 43s
	Train Loss: 3.280 | Train PPL:  26.578
	 Val. Loss: 3.933 |  Val. PPL:  51.048
Epoch: 08 | Time: 0m 43s
	Train Loss: 3.118 | Train PPL:  22.595
	 Val. Loss: 3.859 |  Val. PPL:  47.402
Epoch: 09 | Time: 0m 43s
	Train Loss: 2.991 | Train PPL:  19.914
	 Val. Loss: 3.804 |  Val. PPL:  44.890
Epoch: 10 | Time: 0m 43s
	Train Loss: 2.865 | Train PPL

In [56]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.767 | Test PPL:  43.253 |


## 어떻게 구해지는지.

In [57]:
answer = "good morning"
tokenized = ["good", "morning"]

In [58]:
golden_answer = ["good", "morning", "<eos>"]

In [61]:
vocabulary = {
    "<pad>":0,
    "<sos>":1,
    "<eos>":2,
    "good": 342,
    "morning": 301
}

In [62]:
[vocab.get(t, vocab["<unk>"]) for t in golden_answer]

NameError: ignored

In [ ]:
golden_answer = torch.LongTensor([[342, 301, 2]])

In [ ]:
# (batch_size, sequence_length, vocab_size)
y_pred = torch.randn(1, 3, 1000) # sos, good, morning    GPT도 이렇게 한다..
y_pred.size()

In [ ]:
y_pred.sum(-1) #1000기준으로 다 더하는 것.. 토큰별 어떤 확률분포가 올 것인지.. 합하면 1이되어야하는데 아니다..

In [ ]:
y_pred.view(-1,1000).size()  # 형태변환

In [ ]:
golden_answer.view(-1).size()

In [ ]:
nn.CrossEntropyLoss(ignore_index=0)(y_pred.view(-1,1000), golden_answer.view(-1))  # 갭을 줄여나감..

In [ ]:
import torch.nn.functional as F

F.log_softmax # 듀플리케이트 어지간하면 쓰지말자..

In [ ]:
nn.NLLLoss()(torch.log_softmax(y_pred.view(-1,1000), dim=-1), golden_answer.view(-1)) #예측값에 소프트맥스를 정하기 전에 해야함